In [1]:
import pandas as pd
import numpy as np

In [6]:
import pandas as pd
from pandas import DataFrame

df_tennis = pd.read_csv('PlayTennis.csv')
print("\n Given play tennis data set:\n\n", df_tennis)


 Given play tennis data set:

     Unnamed: 0 PlayTennis   Outlook Temperature Humidity    Wind
0            0         No     Sunny         Hot     High    Weak
1            1         No     Sunny         Hot     High  Strong
2            2        Yes  Overcast         Hot     High    Weak
3            3        Yes      Rain        Mild     High    Weak
4            4        Yes      Rain        Cool   Normal    Weak
5            5         No      Rain        Cool   Normal  Strong
6            6        Yes  Overcast        Cool   Normal  Strong
7            7         No     Sunny        Mild     High    Weak
8            8        Yes     Sunny        Cool   Normal    Weak
9            9        Yes      Rain        Mild   Normal    Weak
10          10        Yes     Sunny        Mild   Normal  Strong
11          11        Yes  Overcast        Mild     High  Strong
12          12        Yes  Overcast         Hot   Normal    Weak
13          13         No      Rain        Mild     High  

In [7]:
df_tennis.keys()[0]

'Unnamed: 0'

In [16]:
def entropy(probs):   
    import math 
    return sum( [-prob*math.log(prob, 2) for prob in probs] ) 
def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0   # = 14 
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances )) 
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO 
    print("\n Classes:",min(cnt),max(cnt)) 
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs))) 
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs))) 
    return entropy(probs) 
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['PlayTennis']) 
total_entropy = entropy_of_list(df_tennis['PlayTennis']) 
print("\n Total Entropy of PlayTennis Data Set:",total_entropy) 


  INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:

 Total Entropy of PlayTennis Data Set: 0.9402859586706309


In [21]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    print("Information Gain Calculation of " ,split_attribute_name)
    df_split=df.groupby(split_attribute_name)
    nobs=len(df.index)*1.0
    df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list,lambda x: len(x)/nobs]})[target_attribute_name]
    df_agg_ent.columns=['Entropy','PropObservations']
    new_entropy=sum(df_agg_ent['Entropy']*df_agg_ent['PropObservations'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy
print('Info-gain for outlook is:'+str(information_gain(df_tennis,'Outlook','PlayTennis')),"\n")
print('\n Info-gain for Humidity is:'+str(information_gain(df_tennis,'Humidity','PlayTennis')),"\n")
print('\nInfo-gain for Wind is:'+str(information_gain(df_tennis,'Wind','PlayTennis')),"\n")
print('\nInfo-gain for Temperature is:'+str(information_gain(df_tennis,'Temperature','PlayTennis')),"\n")

Information Gain Calculation of  Outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:
Info-gain for outlook is:0.2467498197744391 

Information Gain Calculation of  Humidity

 Number of Instances of the Current Sub Class is 7.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.42857142857142855:
 
 Probabilities of Class Yes is 0.5714285714285714:

 Number of Instances of the Current Sub Class is 7.0:

 Classes: 

In [24]:
def id3(df, target_attribute_name, attribute_names, default_class=None): 
    from collections import Counter 
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1: 
        return next(iter(cnt)) 
    elif df.empty or (not attribute_names): 
        return default_class
    else: 
        default_class = max(cnt.keys())
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] # 
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max] 
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr] 
        for attr_val, data_subset in df.groupby(best_attr): 
            subtree = id3(data_subset, 
                        target_attribute_name, 
                        remaining_attribute_names, 
                        default_class) 
            tree[best_attr][attr_val] = subtree 
        return tree

In [25]:
attribute_names=list(df_tennis.columns)
print("List of attributes",attribute_names)
attribute_names.remove('PlayTennis')
print("Predicting attributes",attribute_names)


List of attributes ['Unnamed: 0', 'PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
Predicting attributes ['Unnamed: 0', 'Outlook', 'Temperature', 'Humidity', 'Wind']


In [27]:
from pprint import pprint
tree=id3(df_tennis,'PlayTennis',attribute_names)
print("\n\n The Resultant Decision Tree is:\n")
pprint(tree)
attribute= next (iter (tree))
print("Best Attribute:\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of  Unnamed: 0

 Number of Instances of the Current Sub Class is 1.0:

 Classes: No No
 
 Probabilities of Class No is 1.0:
 
 Probabilities of Class No is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: No No
 
 Probabilities of Class No is 1.0:
 
 Probabilities of Class No is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: No No
 
 Probabilities of Class No is 1.0:
 
 Probabilities of Class No is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Ye

In [29]:
def classify(instance ,tree,default=None):
    attribute=next(iter(tree))
    print("key:",tree.keys())
    print("Atribute:",attribute)
    if instance[attribute] in tree[attribute].keys():
        result= tree[attribute][instance[attribute]]
        print("Instance Attribute:",instance[attribute],"TreeKeys:",tree[attribute].keys())
        if isinstance(result,dict):
            return classify(instance,result)
        else:
            return result
    else:
        return default

In [30]:
df_tennis['predicted']=df_tennis.apply(classify,axis=1,args=(tree,'No'))
print(df_tennis['predicted'])
print('\n Accuracy is:\n'+str(sum(df_tennis['PlayTennis']==df_tennis['predicted'])/(1.0*len(df_tennis.index))))
df_tennis[['PlayTennis','predicted']]

key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 0 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 1 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 2 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 3 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 4 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 5 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: dict_keys(['Unnamed: 0'])
Atribute: Unnamed: 0
Instance Attribute: 6 TreeKeys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
key: d

,PlayTennis,predicted
0,No,No
1,No,No
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,No,No
6,Yes,Yes
7,No,No
8,Yes,Yes
9,Yes,Yes


In [33]:
training_data=df_tennis.iloc[1:-4]
test_data=df_tennis.iloc[-4:]
train_tree =id3(training_data,'PlayTennis',attribute_names)
test_data['predicted2']=test_data.apply(classify,axis=1,args=(train_tree,'Yes'))
                                        
print('\n\n Accuracy is:'+str(sum(test_data['PlayTennis']==test_data['predicted2'])/(1.0*len(test_data.index))))

Information Gain Calculation of  Unnamed: 0

 Number of Instances of the Current Sub Class is 1.0:

 Classes: No No
 
 Probabilities of Class No is 1.0:
 
 Probabilities of Class No is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: No No
 
 Probabilities of Class No is 1.0:
 
 Probabilities of Class No is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 1.0:

 Classes

C:\Users\user\AppData\Local\Temp\ipykernel_2060\638420833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted2']=test_data.apply(classify,axis=1,args=(train_tree,'Yes'))
